In [1]:
# Aluno: ARTUR LUIS BRITO GURJAO   /   Matricula: 20200024903
# Aluno: DEIVISON RODRIGUES JORDAO /   Matricula: 20200023728

In [2]:
!pip install mip

In [3]:
from mip import *
import numpy as np
import re

In [4]:
#Função que lê arquivo TXT e seta as variavéis globais
def read_txt():  #CAMINHO DO ARQUVIVO EM STRING, EXEMPLO: 'projeto.txt'

  global vars  #quantidade de variavéis do modelo
  global rests  #quantidade de restrições
  global list_of_coef_obj #lista com os coeficientes da função objetivo - EX: 4x1 + 5x2 + 8x3  = [4, 5, 8]
  global list_of_coef_rest #lista com os coeficientes das restrições - EX: 5x1 + 1x2 + 4x3 <= 10 = [5, 1, 4, 10]

  caminho = input('Insira o caminho do arquivo txt formatado: ')

  l = []
  with open(caminho) as file: #Onde abre o arquivo TXT e separa por valores e insere numa lista "l"
    for line in file:
        for i in re.findall(r'\d+', line):
            l.append(i)
  
  vars = int(l[0])  #seta nas variáveis globais o vars e o rests pelo primeiro e segundo valor da lista
  rests = int(l[1])
  for i in range(2, 2+vars):  #Um laço que vai do terceiro valor da lista, até a quantidade de variáveis do modelo, que no caso, pega os coeficientes da função objetivo
    list_of_coef_obj.append(int(l[i]))

  count = 0
  aux = []
  for i in range(2+vars, len(l)):  #Um laço que vai da última variável da função objetivo até o final da lista, mas, é utilizado uma lista auxiliar para ir adicionando as restrições
    if count == vars:
      aux.append(int(l[i]))
      list_of_coef_rest.append(aux)
      aux = []
      count = 0
    else:
      aux.append(int(l[i]))
      count += 1

In [5]:
def create_model(vars, rests, list_of_coef_obj, list_of_coef_rest): #Função que recebe quantidade de variáveis, restrições, lista de coeficientes da função objetivo e lista de coeficientes das restrições e retorna um modelo python MIP

  model = Model(sense=MAXIMIZE)  #Parâmetro de maximizar

  x = [model.add_var(var_type="CONTINUOUS",   #Colocamos que as variáveis variam de 0 a 1, de forma contínua
                      lb=0, ub=1, name="x_" + str(i)) for i in range(vars)]

  model.objective = xsum(list_of_coef_obj[i]*x[i]   #Função objetivo vai de um laço i onde pega o indices i da lista de coeficiente vezes a variável criada no x
                           for i in range(vars))

  for i in range(rests):  #Laço que implementa as restrições, pegando os indices j para cada i
      model += xsum(list_of_coef_rest[i][j]*x[j] for j in range(vars)) <= list_of_coef_rest[i][-1]

  return model

In [6]:
def solver(model):    #Função solver que retorna um dicionário, nele há 2 chaves, objective é onde fica o valor da função objetivo, vars é onde fica uma lista com as variáveis
  model.optimize()
  params = {}
  params["objective"] = model.objective_value
  params["vars"] = model.vars

  return params

In [7]:
def bound(model):    #Função que retorna string de acordo com o modelo
  global primal
  flag_solver = solver(model)
  count_int = 0

  if flag_solver["objective"] == None:   #Se a função objetivo do modelo for None, é inviável
    return 'INVIABILIDADE'

  for i in flag_solver["vars"]:  #Aqui faz um contador de inteiros no count_int
    if i.x.is_integer():
      count_int += 1
  
  if count_int == len(flag_solver["vars"]): #Se a quantidade de inteiros nas variáveis for igual a quantidade de variáveis, são todos inteiros então retorna integralidade
    return 'INTEGRALIDADE'

  if flag_solver["objective"] <= primal:   #Aqui retorna limite se a função objetivo for menor que o primal, sendo limitante
    return 'LIMITE'
  return 'FRACIONÁRIO'    #Aqui retorna fracionário se não for nenhuma das acimas, então é viável o modelo

In [8]:
def find_nearest(array, value):   #Função que recebe uma lista de variável e um valor, busca na lista a variável mais próxima do valor e retorna o indíce dela(idx)
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [9]:
def branch(model, values_solution): #Função branch, recebe o modelo e a lista de variáveis do modelo, que ramifica e retorna uma lista com os 2 modelos(restrições x==0 e x==1)

  var_branch = values_solution[find_nearest([i.x for i in values_solution], 0.5)]   #nessa var_branch, pegamos o valor da lista de variáveis que tem o valor mais próximo de 0.5, através do indice que é retornado da função find_nearest

  model_0 = model.copy()   #Nó que tem restrição var == 0
  model_0 += var_branch == 0

  model_1 = model.copy()   #Nó que tem restrição var == 1
  model_1 += var_branch == 1

  return (model_0, model_1)

In [10]:
def branch_and_bound(model):  #Função branch_and_bound que faz todo o preço de adicionar e remover da lista de nós
  global optimal_model
  global primal #Utilizando apenas o primal
  nodes = [model]  #Fila

  while nodes != []:   #Enquanto a lista estiver com modelos, ele continua
    model_solver = solver(nodes[0])
    flag_bound = bound(nodes[0])    #Aqui utilizamos a função bound que tem como papel retornar se deve podar a ramificação, a única string retornada que não é podada é a 'FRACIONÁRIO'
    if flag_bound == 'INVIABILIDADE' or flag_bound == 'LIMITE':   
      nodes.pop(0)   #Se for inviável ou limitante, apenas removemos o atual modelo da lista de modelos(nó)
    elif flag_bound == 'INTEGRALIDADE':    #Se for podada por integralidade, verificamos se o valor da função objetivo é maior ou igual que o primal, se for, o modelo ótimo é substituido pelo modelo atual, o primal também é atualizado
      if model_solver["objective"] > primal:
        optimal_model = nodes[0]
        primal = model_solver["objective"]
      nodes.pop(0)
    elif flag_bound == 'FRACIONÁRIO':   #Se for fracionário(viável), faremos duas restrições, utilizando a função bound, que retorna uma lista com os modelos já com as restrições e adiciona na lista com os nós, após isso é removido o atual modelo
      flag_branch = branch(nodes[0], model_solver["vars"])
      nodes.append(flag_branch[0])
      nodes.append(flag_branch[1])
      nodes.pop(0)

In [11]:
def main():  #Função que faz o processo de chamar as outras funções
  read_txt()

  run = create_model(vars, rests, list_of_coef_obj, list_of_coef_rest)

  branch_and_bound(run)
  model_solved = solver(optimal_model)

  print("Variables:")
  for i in model_solved["vars"]:
    print(i.name, ' = ', i.x)
  print("Objective function:")
  print('Z = ', model_solved["objective"])

In [12]:
#VARIAVEIS GLOBAIS

primal = 0 #inferior
optimal_model = None
vars = 0
rests = 0
list_of_coef_obj = []
list_of_coef_rest = []

#CHAMADO DA FUNÇÂO MAIN
main()

Insira o caminho do arquivo txt formatado: teste_po.txt
Variables:
x_0  =  0.0
x_1  =  0.0
x_2  =  0.0
x_3  =  0.0
x_4  =  1.0
x_5  =  1.0
x_6  =  0.0
Objective function:
Z =  20.0
